# CoC to Census Tract Mapping
In this section, we will associate each Continuum of Care (CoC) with relevant census data, such as census level demographics, income, and cost of living data. To do so, we will use shape files of each CoC and US Census Bureau tracts. Note that tracts may change on an annual basis; therefore, these data must be generated for each year.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import numpy as np
from shapely.geometry import Point
from urllib.request import urlretrieve
from urllib.parse import urlparse
from utils import *
from parameters import *
import openpyxl
from concurrent.futures import ThreadPoolExecutor

## Algorithm
CoCs can contain many US Census Bureau tracts. For each tract constituting a CoC, we note the fraction of the total tract encompassed in the CoC. Tracts of which less than 1% of the total tract area is within the CoC are omitted.

Algorithmically, this is done by performing intersections between shape files and then calculating the total area of overlap after converting to an area-preserving coordinate reference system. The data are stored in dictionaries for each state.

In [ ]:
census_api_key = "85e4d04ebdc75be192b6cd5444f6dc9970a95666"

# summary_2024 = create_coc_summary(2011, census_api_key)


In [ ]:

years = range(2011, 2025)

with ThreadPoolExecutor(max_workers=10) as parallel_executor:
    futures = []
    for year in years:
        futures.append(parallel_executor.submit(create_coc_summary, year, census_api_key))

with pd.ExcelWriter('coc_summary_4.xlsx') as writer:
    for yr, fut in zip(years, futures):
        try:
            df = fut.result()
            df.to_excel(writer, sheet_name=str(yr), index=True)
        except Exception as e:
            print(f"Year {yr}: {e}")